In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

def implicit_full_matrix(N, L, initial_value, left_bc, rightt_bc, c, rho, k_function, delta_t, t_n):
    
    """
    N : Number of nodes (including boundaries)
    L : Length of domain
    initial_values : .....
    c : specific heat of medium (scalar constant)
    rho : density of medium (scalar constant)
    K : thermal conductivity of medium (function of scalar variable, u)
    left_bc
    right_bc
    delta_t
    t_n
    """
    
    # System of equations
    A = np.ndarray( (N,N), dtype=np.float64)
    f = np.ndarray( (N), dtype=np.float64)

    # Create solution vector
    u_old = np.ndarray( (N), dtype=np.float64)
    u_new = np.ndarray( (N), dtype=np.float64)
    
    # Distance between nodes
    h = L/(N-1)
    
    # Set initial values
    u_new.fill(initial_value)

[[ 0.]]
[  6.89835425e-310]
